In [1]:
!pip install ray[rllib]

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import gymnasium as gym
from gymnasium import spaces, vector
import numpy as np
from scipy.stats import binom, nbinom, beta, poisson, gamma, norm, geom
import random
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial

2024-05-11 10:49:30,936	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!


In [3]:
!pip install pokerlib
from pokerlib.enums import Rank, Suit, Hand
from pokerlib import HandParser
import random

Defaulting to user installation because normal site-packages is not writeable


# These are attempts for 2

In [4]:
import random
from collections import Counter
from itertools import combinations

import gymnasium as gym
from gym import spaces
from ray import tune

from gymnasium import spaces, vector
import numpy as np
from scipy.stats import binom, nbinom, beta, poisson, gamma, norm, geom
import ray
from ray import tune, air
from ray.rllib.algorithms.ppo import PPOConfig
from functools import partial
# Card and Deck definitions
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return f"{self.rank}{self.suit}"

class Deck:
    ranks = "23456789TJQKA"
    suits = "SHDC"

    def __init__(self):
        self.cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
        random.shuffle(self.cards)

    def draw(self, count=1):
        return [self.cards.pop() for _ in range(min(count, len(self.cards)))]

# Game Environment
class CardGameEnv(gym.Env):
    metadata = {'render.modes': ['human']}
    hand_values = {
        'Straight Flush': 800, 'Four of a Kind': 420, 'Full House': 160,
        'Flush': 140, 'Straight': 120, 'Three of a Kind': 90,
        'Two Pair': 40, 'Pair': 20, 'High Card': 5
    }

    def __init__(self, config=None):
        super(CardGameEnv, self).__init__()
        self.deck = Deck()
        self.hand = self.deck.draw(8)
        self.discards = 3
        self.plays = 4
        self.total_score = 0
        self.action_space = spaces.Discrete(len(self.hand) + 1)  # Choose how many cards to play or discard
        self.observation_space = spaces.MultiDiscrete([13] * 8)  # Represent card ranks in hand

    # Other methods remain unchanged...

    def step(self, action):
        assert self.action_space.contains(action), f"Invalid Action: {action}"
        cards_to_play = random.sample(self.hand, action)
        self.play_cards(cards_to_play)
        reward = self.evaluate_hand(cards_to_play)
        # self.hand += self.deck.draw(action)
        self.total_score += reward
        done = self.plays == 0
        return self._get_obs(), reward, done, {}

    def reset(self):
        self.deck = Deck()
        self.hand = self.deck.draw(8)
        self.total_score = 0
        self.plays = 4
        self.discards = 3
        return self._get_obs()


    def _get_obs(self):
        # Initialize an array to hold the observations
        obs = np.zeros(8, dtype=np.int64)

        # Iterate over the hand of cards
        for i, card in enumerate(self.hand):
            # Get the index of the card's rank in the ranks list
            rank_index = self.deck.ranks.index(card.rank)
            # Set the observation value corresponding to the card's rank
            obs[i] = rank_index

        return obs


    def render(self, mode='human', close=False):
        print("Current hand:", self.hand)
        print("Score:", self.total_score)

    def play_cards(self, cards):
        for card in cards:
            self.hand.remove(card)

    def evaluate_hand(self, cards):
        if len(cards) < 5:
            return 0  # Less than 5 cards cannot form any of the hands listed (except for advanced rules)

        ranks = [card.rank for card in cards]
        suits = [card.suit for card in cards]
        rank_counts = Counter(ranks)
        suit_counts = Counter(suits)
        is_flush = max(suit_counts.values()) == 5
        sorted_ranks = sorted(ranks, key=lambda x: self.deck.ranks.index(x))
        rank_sequence = [self.deck.ranks.index(rank) for rank in sorted_ranks]
        is_straight = all(rank_sequence[i] + 1 == rank_sequence[i + 1] for i in range(len(rank_sequence) - 1))

        if is_flush and is_straight:
            return self.hand_values['Straight Flush']
        if max(rank_counts.values()) == 4:
            return self.hand_values['Four of a Kind']
        if sorted(rank_counts.values()) == [2, 3]:
            return self.hand_values['Full House']
        if is_flush:
            return self.hand_values['Flush']
        if is_straight:
            return self.hand_values['Straight']
        if max(rank_counts.values()) == 3:
            return self.hand_values['Three of a Kind']
        if len([rank for rank, count in rank_counts.items() if count == 2]) == 2:
            return self.hand_values['Two Pair']
        if len([rank for rank, count in rank_counts.items() if count == 2]) == 1:
            return self.hand_values['Pair']
        return self.hand_values['High Card']

# Training with RLlib using Config
config = (PPOConfig()
          .environment(CardGameEnv)
          .framework('torch')
          .training(gamma=1)
          .rollouts(num_rollout_workers=3)
)
stop = {"timesteps_total": 10}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

res = tuner.fit()

2024-05-11 10:49:36,044	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1098044) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1098044) 2024-05-11 10:49:39,162	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con

In [5]:
import numpy as np
import gymnasium as gym
from gym import spaces
from gym.utils import seeding

class CardGameEnv(gym.Env):
    def __init__(self, seed = None):
        self.action_space = spaces.MultiBinary(8)  # Binary array to represent discarding individual cards
        self.observation_space = spaces.Box(low=0, high=51, shape=(8, 2), dtype=np.int16)
        self.suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
        self.values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
        self.deck =  [(value, suit) for value in range(1, 14) for suit in range(4)]
        self.seed()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def reset(self, *, seed=None, options=None):
        self.hand = self.draw_hand(8)  # Number, Suit
        # print(self.hand)
        self.discards = 3
        self.plays = 4
        self.total_reward = 0
        return np.array(self.hand), {}

    def draw_hand(self, num_cards_to_draw):
        cards = []
        for i in range(num_cards_to_draw):
            card = random.choice(self.deck)
            # Remove the selected card from available deck
            self.deck.remove(card)
            cards.append(card)
        # print("Cards remaining:", len(self.deck))
        return cards

    # def step(self, action):
    #     print("Startinghand", self.hand)
    #     assert len(action) == 8

    #     discards = [idx for idx, val in enumerate(action) if val == 1]
    #     num_discards = len(discards)
    #     while len(discards) > 5:
    #         discards.pop()
    #         num_discards = len(discards)
    #     print("Discards", discards)
    #     assert 0 <= num_discards <= 5  # Agent can discard at most 5 cards and at least 0 cards
    #     assert num_discards < 8  # Agent must keep at least one card

    #     if num_discards > 0:  # Agent chose to discard specific cards
    #         num_new_cards = 8 - num_discards
    #         new_cards = self.draw_hand(num_new_cards)
    #         # Calculate remaining indices ensuring they are within the range of self.hand
    #         remaining_indices = [0,1,2,3,4,5,6,7]
    #         for i in discards:
    #             remaining_indices.remove(i)
    #         print(remaining_indices)
    #         print("hand", self.hand)
    #         self.hand = [self.hand[idx] for idx in remaining_indices] + new_cards

    #     done = self.plays == 0
    #     return np.array(self.hand), self.total_reward, done, {}

    def step(self, action):
        # print("Startinghand", self.hand)
        assert len(action) == 8

        discards = [idx for idx, val in enumerate(action) if val == 1]
        num_discards = len(discards)
        # Ensure that number of discards is at most 5
        while len(discards) > 5:
            discards.pop()
            num_discards = len(discards)
        # print("Discards", discards, len(discards))
        assert 0 <= num_discards <= 5  # Agent can discard at most 5 cards and at least 0 cards
        assert num_discards < 8  # Agent must keep at least one card

        if num_discards > 0:  # Agent chose to discard specific cards
            num_new_cards = num_discards
            new_cards = self.draw_hand(num_new_cards)
            # Calculate remaining indices ensuring they are within the range of self.hand
            remaining_indices = [i for i in range(8) if i not in discards]
            # print("Remaining Indices:", remaining_indices)
            # print("Hand before discarding:", self.hand)
            self.hand = [self.hand[idx] for idx in remaining_indices] + new_cards
            # print("Hand after discarding:", self.hand)

        self.discards -= 1
        if self.discards == 0 or self.plays == 0:
            done = True
        else:
            done = False
        
        reward = self.calculate_hand_value(self.hand)
        self.total_reward = reward

        return np.array(self.hand), self.total_reward, done, {}


    def calculate_hand_value(self, hand):
        # Generate all combinations of 5 cards
        combinations_5 = combinations(hand, 5)
        best_reward = 0
        for combo in combinations_5:
            card_values = sorted([card[0] for card in combo])
            card_suits = [card[1] for card in combo]
            counts = np.bincount(card_values)
            num_unique_cards = np.count_nonzero(counts)
            is_flush = len(set(card_suits)) == 1

            if num_unique_cards == 2:
                if 4 in counts:
                    reward = 420  # Four of a Kind
                else:
                    reward = 160  # Full House
            elif num_unique_cards == 3:
                if 3 in counts:
                    reward = 90  # Three of a Kind
                else:
                    reward = 40  # Two Pair
            elif num_unique_cards == 4:
                reward = 20  # Pair
            elif num_unique_cards == 5:
                is_straight = all(card_values[i] == card_values[i-1] + 1 for i in range(1, len(card_values)))
                if is_straight:
                    if is_flush:
                        reward = 800  # Straight Flush
                    else:
                        reward = 120  # Straight
                elif is_flush:
                    reward = 140  # Flush
                else:
                    reward = 5  # High Card
            
            if reward > best_reward:
                best_hand = combo
                best_reward = reward
        # print(best_hand)
        return best_reward

# Test the environment
env = CardGameEnv()
obs = env.reset()
total_reward = 0
done = False
while not done:
    action = env.action_space.sample()  # Random action for testing
    obs, reward, done, _ = env.step(action)
    total_reward = reward
print("Reward:", total_reward)

# env.calculate_hand_value([(9, 2), (5, 2), (7, 1), (0, 2), (6, 2), (1, 0), (11, 0), (3, 0)])


Reward: 20


In [6]:
if ray.is_initialized():
  ray.shutdown()

runtime_env = {"py_modules": ["../.."]}
ray.init(runtime_env=runtime_env)

2024-05-11 10:49:48,096	INFO worker.py:1642 -- Started a local Ray instance.
2024-05-11 10:49:48,189	INFO packaging.py:518 -- Creating a file package for local directory '../..'.
2024-05-11 10:49:48,275	INFO packaging.py:346 -- Pushing file package 'gcs://_ray_pkg_a5ba02e7ebc4b7e1.zip' (9.77MiB) to Ray cluster...
2024-05-11 10:49:48,320	INFO packaging.py:359 -- Successfully pushed file package 'gcs://_ray_pkg_a5ba02e7ebc4b7e1.zip'.


Python version:,3.10.12
Ray version:,2.7.0


In [7]:
config = (PPOConfig()
          .environment(CardGameEnv)
          .framework('torch')
          .training(gamma=1)
          .rollouts(num_rollout_workers=3)
)
stop = {"timesteps_total": 10}

tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=air.RunConfig(stop=stop),
)

res = tuner.fit()


2024-05-11 10:49:49,519	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=1099329) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1099329) 2024-05-11 10:49:53,480	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_con